Training, Validation and testing dataset were labeled by batch manually

In [1]:
import sys
sys.path.append('/workspace/Documents')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Diffusion_denoising_thin_slice.functions_collection as ff

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '/mnt/camca_NAS/denoising/Patient_lists'
patient_sheet = pd.read_excel(os.path.join('/mnt/camca_NAS/denoising/','Patient_lists', 'fixedCT_static.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
print('patient sheet len: ', len(patient_sheet))

patient sheet len:  100


In [10]:
# # shuffle the patient list and split into 6 batches
# A = np.arange(0,100)
# np.random.shuffle(A)
# num_batches = 6
# batches = np.array_split(A, num_batches)

# # Print the batches
# for i, batch in enumerate(batches):
#     print('batch len: ', len(batch))
#     print(f"Batch {i+1}: {batch}")


batch len:  17
Batch 1: [31 96 61 62 41 29 45 83 32 23 59 10 16 15 33 85  2]
batch len:  17
Batch 2: [94 14 93 82 30 98 69 19 26 42 50 90 17 55 12  1 91]
batch len:  17
Batch 3: [76 67 73 18 74 13 95 39 40 57 66 43  8 64 97 56 53]
batch len:  17
Batch 4: [28 21 34 36 88 35 81 65 49 46 92 89 77  3  4 84  9]
batch len:  16
Batch 5: [ 5 71 51 80 11 78 79 68  7  0 38 47 22 99 63 58]
batch len:  16
Batch 6: [48 75 24 27 54 37 20 70 60 44 72 87  6 86 25 52]


In [11]:
batch_column = []
for i in range(0,len(patient_sheet)):
    for j in range(0,num_batches):
        if i in batches[j]:
            batch_column.append(j)
            break
patient_sheet['batch'] = batch_column

In [12]:
patient_sheet_shuffled = patient_sheet.iloc[A].reset_index(drop=True)
patient_sheet_shuffled.to_excel(os.path.join(data_path, 'fixedCT_static_shuffled_batched.xlsx'), index=False)

# build list for simulations

In [3]:
patient_sheet = pd.read_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_shuffled_batched.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
noise_types = ['gaussian','possion']
simulation_num = 2

link = 'NAS' # 'local' or 'NAS'
if link == 'local':
    data_path1 = '/workspace/Documents/Data/denoising'
elif link == 'NAS':
    data_path1 = '/mnt/camca_NAS/denoising/Data'

for noise_type in noise_types:
    results = []
    for i in range(0, len(patient_sheet)):
        patient_id = patient_sheet['Patient_ID'][i]
        patient_subid = patient_sheet['Patient_subID'][i]
        batch = patient_sheet['batch'][i]
        print(f"Processing patient {patient_id} {patient_subid}...")

        # simulation_files = ff.find_all_target_files([noise_type+'_random*'],os.path.join(data_path1, 'simulation/',patient_id,patient_subid))
        # simulation_files = ff.sort_timeframe(simulation_files,0,'_','')
        # assert len(simulation_files) == simulation_num, f"Simulation files number is not {simulation_num}!"

        for n in range(0,simulation_num):
            simulation_file = os.path.join(data_path1,'simulation/',patient_id,patient_subid,noise_type+'_random_'+str(n), 'recon.nii.gz')
            ground_truth_file = os.path.join(data_path1,'fixedCT/',patient_id,patient_subid,'img_thinslice_partial.nii.gz')

            # if n == 0:
            #     simulation_img = nb.load(simulation_file).get_fdata()
            #     ground_truth_img = nb.load(ground_truth_file).get_fdata()
            #     assert simulation_img.shape == ground_truth_img.shape, "Simulation and ground truth images have different shapes!"

            results.append([batch,patient_id, patient_subid, n,simulation_file, ground_truth_file])#, simulation_img.shape])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'Patient_subID', 'random_num', 'noise_file', 'ground_truth_file'])#, 'image_shape'])
        df.to_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_simulation_train_test_' + noise_type + '_' + link +'.xlsx'), index=False)





Processing patient 00139437 0000258390...
Processing patient 00019599 0000029506...
Processing patient 00148611 0000455363...
Processing patient 00173704 0000455358...
Processing patient 00030597 0000455473...
Processing patient 00214862 0000455530...
Processing patient 00004038 0000455420...
Processing patient 00214775 0000455224...
Processing patient 00214865 0000455504...
Processing patient 00214905 0000455573...
Processing patient 00214815 0000455367...
Processing patient 00214925 0000455633...
Processing patient 00214918 0000455613...
Processing patient 00214919 0000455609...
Processing patient 00214864 0000455532...
Processing patient 00010431 0000019300...
Processing patient 00214935 0000455654...
Processing patient 00010536 0000455747...
Processing patient 00058992 0000455611...
Processing patient 00010795 0000455825...
Processing patient 00171178 0000455242...
Processing patient 00093479 0000455522...
Processing patient 00008527 0000455795...
Processing patient 00154227 000045

# build list for distilled model

In [3]:
patient_sheet = pd.read_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_shuffled_batched.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
simulation_num = 2

link = 'local' # 'local' or 'NAS'
if link == 'local':
    data_path1 = '/workspace/Documents/Data/denoising'
# elif link == 'NAS':
#     data_path1 = '/mnt/camca_NAS/denoising/Data'

results = []
for i in range(0, len(patient_sheet)):
    patient_id = patient_sheet['Patient_ID'][i]
    patient_subid = patient_sheet['Patient_subID'][i]
    batch = patient_sheet['batch'][i]
    print(f"Processing patient {patient_id} {patient_subid}...")


    for n in range(0,simulation_num):
        simulation_file = os.path.join(data_path1,'pred_images/',patient_id, patient_subid, 'random_'+str(n), 'condition_img.nii.gz')

        ground_truth_file = os.path.join(data_path1,'pred_images/',patient_id,patient_subid,'random_'+str(n), 'pred_img_scans20.nii.gz')

        results.append([batch,patient_id, patient_subid, n,simulation_file, ground_truth_file])#, simulation_img.shape])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'Patient_subID', 'random_num', 'noise_file', 'ground_truth_file'])#, 'image_shape'])
        df.to_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_distilled_model_train_test_' + link +'.xlsx'), index=False)





Processing patient 00139437 0000258390...
Processing patient 00019599 0000029506...
Processing patient 00148611 0000455363...
Processing patient 00173704 0000455358...
Processing patient 00030597 0000455473...
Processing patient 00214862 0000455530...
Processing patient 00004038 0000455420...
Processing patient 00214775 0000455224...
Processing patient 00214865 0000455504...
Processing patient 00214905 0000455573...
Processing patient 00214815 0000455367...
Processing patient 00214925 0000455633...
Processing patient 00214918 0000455613...
Processing patient 00214919 0000455609...
Processing patient 00214864 0000455532...
Processing patient 00010431 0000019300...
Processing patient 00214935 0000455654...
Processing patient 00010536 0000455747...
Processing patient 00058992 0000455611...
Processing patient 00010795 0000455825...
Processing patient 00171178 0000455242...
Processing patient 00093479 0000455522...
Processing patient 00008527 0000455795...
Processing patient 00154227 000045